# Yelp Reviews Rating Prediction

## 1. Introduction
In this project, we aim to find out what makes a review positive or negative, and predict rating of businesses listed in the Yelp dataset based on review text and other variables, and check how the sentiment varied over geographical location, time and other factors. To find the combination of best machine learning technique and feature extraction method to solve the problem, linear regression and different classification techniques such as Naïve Bayes, LSTM and SVM were used.

## 2. Exploratory Analysis


In the training set, there are 1,546,379 observations, which containing specific information about a review, like its date, text, star, location and categories. In the testing and validation data, there are 1,016,664 elements without ratings.


### 2.1 Basic Statistics and Properties

First, we check the training data and find that there is no missing value. But based on the initial data, there are 45 records happened in a city named Henderson which should be located in North America, have longitude 115.0858. We consider this problem as a clerical error and use -115.0858 instead.

Then we aggregate the training data in order to find the top cities and top categories. The top cities like Las Vegas concentrate the majority of the reviews and mostly located in North America. The most frequently reviewed categories are Restaurants, American and Bars.

<img src="image/Average_Rating_by_Region.png" width="60%">

Moreover, we analyse what is the average rating stars, given the length of reviews. As the review lengths increase, only a small subset of it has 500 or more tokens. There's a high concentration and rating decreases before 300. And after that even though the dispersion is significant, the trend is still noticeable. This may because number of reviews decreases and rating variance increases quickly.

|   Overall Review Lengths   |         Length by Stars         |
| :------------------------: | :-----------------------------: |
| ![](image/Length_Dist.png) | ![](image/Rating_by_Length.png) |

From 2005, Yelp has had incredible growth and an annual pattern in the number of reviews in each of the five rating brackets. The fact is, early Yelp had mostly 4-star and 5-star reviews. However, the behavior from 2010 onward is interesting: the proportions of both 1-star and 5-star reviews increase over time.

|        Rating Proportion over Time         |          Review Count over Time          |
| :----------------------------------------: | :--------------------------------------: |
| ![](image/Rating_Proportion_over_Time.png) | ![](image/Yelp_Review_Count_by_Time.png) |

### 2.2 Text Processing

We first change negative suffice to a separate 'not', e.g. from 'wouldn't' to 'would not'. Then we apply standard technique to tokenize the sentence into words bags and remove punctuation marks. For stop-words, we mantually keep some of them, such as not, he, she and etc. The following steps are stemming, lemmatizing and convert the text to lower case, which are applied to reduce dimension.

Also, we detect that there are some reviews written in French and Japanese. After the steps mentioned above, these records will be coded as missing values. So we use the BlobText package to search and translate these foreign language.

### 2.3 Feature Extraction

Most of the elements in word dictionary have low frequencies. We firstly select the 10000 most frequent words and then use following formula to calculate their rating variance:

<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"> </script>
$$variance(t) = \frac{\sum_i [\text{# of }t\text{ in }i\text{ star reviews}*i-mean(t)]^2}{\text{# of }t \text{ in all of the reviews}}$$

Words with high variance will be deleted. The same method is applied into bigram features. In total, about 5000 unigram and 10000 bigram features are extracted from the reviews. Two sparse matrix of features for unigram and bigram features based on TF-IDF algorithm are created separately. The following wordclouds shows the top unigram, bigram and trigram features for 5-star reviews.

|       Unigram       |       Bigram       |       Trigram       |
| :-----------------: | :----------------: | :-----------------: |
| ![](image/Uni5.png) | ![](image/Bi5.png) | ![](image/Tri5.png) |

## 3. Models
We trained several models based on the data. Two of them have relative nice performance - SVM with TF-IDF and LSTM neural network with linear regression to predict the stars rating given the text of a review and other factors. The predictors we have are:
1. Unigram and bigram features from review texts
2. Categories
3. City
4. Business name

### 3.1 Support Vector Machine with TF-IDF

#### 3.1.1 TF-IDF

TF-IDF is the product of Term Frequency and Inverse Document Frequency. Term Frequency tf(t,d) is the number of times that term t occurs in document d, While Inverse Document Frequency idf(t,D) is the logarithmically scaled inverse fraction of the documents that contain the word.

#### 3.1.2 Support Vector Machine

Keeping all input unigram and bigram features, we make a sparse matrix by TF-IDF method and experiment using scikit’s SVM with linear kernel, and got an RMSE of 0.73 on Kaggle.

### 3.2 LSTM + Linear Regression

#### 3.2.1 Long-Short Term Memory
Given the characteristics of the reviews presented in previous sections, we build up a vocabulary of 5k top featured words and a maximum review length of 400, which provide coverage for 95% of more of the cases.

Moreover, given the higher proportion of 5-star ratings, we randomly take 150k samples for each of the 5 rating brackets, totally 600k, to balance the training dataset. For the embeddings, we choose to use the 100-dimension vectors, as bigger vectors would increase the cost of training the models.

For the structure of the neural network itself, the trends of accuracy and loss do not improve much after the 6th epoch.
So we finally experiment a LSTM design with 1 layer, 0.2 dropout rate and 6 epochs.


|      Accuracy      |        Loss         |
| :----------------: | :-----------------: |
| ![](image/ACC.png) | ![](image/LOSS.png) |

#### 3.2.2 Linear Regression

Based on the prior analysis, we want to add more variables like length of reviews and number of exclamation marks into the model to test whether they have an impact on ratings. A simple linear model is performed on 1,000k training data and the rest part is treated as test data. After rounding the predictions, we can see that the model performs better at both extremes of the rating scale: 1 and 5-star ratings.

|        Residuals        | QQ Plot of Residuals |        Confusion Matrix         |
| :---------------------: | :------------------: | :-----------------------------: |
| ![](image/Residual.png) |  ![](image/QQ.png)   | ![](image/Confusion_Matrix.png) |

### 3.3 Model Comparison

|        Model         | Ridge | k-NN | Random_Forest | SLR | SVR  | SVM    | LSTM |      LSTM+Linear |
| ------------------:  | ----: | ---: | ------------: | ---:|---:  |----:   | ---: | ---------------: |
| Size of Training Set |  All  |  100k |     200k       | All | 10k | All    | 600k  |           1,000k |
| Features             |  Uni  |  Uni |     Uni       | Uni | Uni  | Uni&Bi | Uni  |              Uni |
|         RMSE         |  1.71 | 1.49 |    0.81       | 0.79|0.75  | 0.73   | 0.66 |              0.62|

## 4. Model Evaluation

### 4.1 Strengths

1. Accuracy: This final model LSTM + Linear Regression is quite accurate and performances well with RMSE value of around 0.62.
2. Robustness: The more training data we use to train the model, the better the results will be. The diagnostic results of our model show its reliablity and there is no evidence suggest our model has overfitting issue.


### 4.2 Weaknesses

1. Time consuming: Our final model has been trained 10 rounds and each round takes for more than 3 hours, so overall our model can’t make predictions in a very short time period.
2. Feature Selection: We only consider the meaningful single word in the reviews, which may loss information of bigram features. However, our SVM model combines the bigram which greatly elevates its performance. We average the results of SVM and LSTM+Linear models to produce our final predcition, the RMSE on Kaggle reduces by 0.007.
3. Typos: Spell checking is not applied into text cleaning process due to the time limitation.
4. Dimension Reduction: The way we used to reduce dimension is a little bit simple. Future analysis may consider more complicated algorithms like SVD.

## 5. Conclusion
We can decide the positive or negative of the review by the result of our model prediction which is also very precise.

To propose a prediction model to predict the ratings of reviews, firstly, we remove the meaningless part in the text reviews and create a sparse matrix based on the extracted unigrams. Then we extract important bigrams and create a matrix as well. For the model section, we achieve an accurate predict model through the combination of LSTM using 100-dimension vectors and simple linear model. We also consider some other features in our model such as categories, city, longitude and latitude to improve our model’s accuracy. Our model can perform better with tuned parameters and more training data.

## 6. Further Proposals
From the geographical map, rating performance differs among city. Also, the average stars of chain stores is a little higher than that of local stores. Besides, among several main categories, 'brunch' is better than others. Therefore, investing a branch chain restaurant in specific city may be a reasonable strategy.

### Reference

1. Support vector machines: theory and applications[M]. Springer Science & Business Media, 2005.

2. Sundermeyer M, Schlüter R, Ney H. LSTM neural networks for language modeling[C]//Thirteenth Annual Conference of the International Speech Communication Association. 2012.

### Contributions
|    __Name__    | __Contribution__ |
| -------------: | :--------------- |
|   Shaodong Wang  | Worked on regression models, reducted dimension of sparse matrix.      |
|   Wenjin Li    | Cleaned the data, built up the LSTM neural network model.       |
|   Jia Liu  | Made the wonderful plots and charts, prepared the slides.|
|   Jiayin Wang  | Made exploratory analysis, compiled the jupyter notebook.|